In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import mne
import keras
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import random
from torch import optim
import argparse
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import torch.nn.functional as F
from os import walk

In [3]:
DEVICE = torch.device('cuda:0')

In [4]:
def pad(rawdata, seglength):
    pad_length = -rawdata.size()[0] % seglength
    padded = np.concatenate((rawdata, torch.zeros((pad_length, rawdata.size()[1]))), 0)
    return padded

In [5]:
def to_epochs(rawdata, seg_length):
    data = pad(rawdata, seg_length)
    """
    split = torch.Tensor([])
    for element in torch.split(data, seg_length):
        split = torch.cat((split, element))
        """
    split = np.array(np.split(data, seg_length))
    return split.reshape(-1, seg_length, 20)

In [6]:
def to_gpu(x, on_cpu=False, gpu_id=None):

    if torch.cuda.is_available() and not on_cpu:
        x = x.cuda(gpu_id)
    return x

In [7]:
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(d, device) for d in data]
    return data.to(device, non_blocking=True)

In [8]:
def PNES(path, train_ratio = 0.75):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    pnes = []
    for i in range(len(names)):
        pnes.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))
    
    pnes_epochs = []
    for patient in pnes:
        pnes_epochs.append(to_epochs(patient))
    
    random.shuffle(pnes_epochs)
    idx = (np.ceil(0.75 * len(pnes_epochs))).astype(int)
    train = pnes_epochs[0:idx]
    test = pnes_epochs[idx:]
    pnes_train_labels = np.ones(len(train)).reshape(-1) * 3
    pnes_test_labels = np.ones(len(test)).reshape(-1) * 3
    return train, test, pnes_train_labels, pnes_test_labels

In [9]:
def FOCAL(path, train_ratio = 0.75, seglength = 9000):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    focal = []
    for i in range(len(names)):
        focal.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))
    
    focal_epochs = []
    for patient in focal:
        focal_epochs.append(to_epochs(patient, seglength))
    
    random.shuffle(focal_epochs)
    idx = (np.ceil(train_ratio * len(focal_epochs))).astype(int)
    train = focal_epochs[0:idx]
    test = focal_epochs[idx:]
    focal_train_labels = np.ones(len(train)).reshape(-1) * 0
    focal_test_labels = np.ones(len(test)).reshape(-1) * 0
    return train, test, focal_train_labels, focal_test_labels

In [10]:
def IGE(path, train_ratio, seglength):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    ige = []
    for i in range(len(names)):
        ige.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))
    
    ige_epochs = np.array([])
    for patient in ige:
        ige_epochs = np.concatenate((ige_epochs, to_epochs(patient, seglength)), axis = 0)
    
    random.shuffle(ige_epochs)
    idx = (np.ceil(train_ratio * len(ige_epochs))).astype(int)
    train = ige_epochs[0:idx]
    test = ige_epochs[idx:]
    ige_train_labels = np.ones(len(train)).reshape(-1)
    ige_test_labels = np.ones(len(test)).reshape(-1)
    return train, test, ige_train_labels, ige_test_labels

In [18]:
def TLE(path, train_ratio, seglength):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    temporal = []
    for i in range(len(names)):
        temporal.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))

    temporal_epochs = to_epochs(temporal[0], seglength)
    for patient in temporal:
        temporal_epochs = np.concatenate((temporal_epochs, to_epochs(patient, seglength)), axis = 0)
    
    np.random.shuffle(temporal_epochs)
    idx = (np.ceil(0.75 * len(temporal_epochs))).astype(int)
    train = temporal_epochs[0:idx]
    test = temporal_epochs[idx:]
    train_labels = np.ones(len(train)).reshape(-1) * 4
    test_labels = np.ones(len(test)).reshape(-1) * 4
    

    return train, test, train_labels, test_labels

In [12]:
path = 'D:\EEG\TLE'
seglength = 9000
names = []
for path, _, name in os.walk(path):
    names.append(name)
    names = np.squeeze(names)
    
temporal = []
for i in range(len(names)):
    temporal.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))

In [13]:
temporal_epochs = np.zeros(np.shape(to_epochs(temporal[0], seglength)))

In [15]:

    
temporal_epochs = to_epochs(temporal[0], seglength)
for patient in temporal:
    temporal_epochs = np.concatenate((temporal_epochs, to_epochs(patient, seglength)), axis = 0)
    
np.random.shuffle(temporal_epochs)
idx = (np.ceil(0.75 * len(temporal_epochs))).astype(int)
train = temporal_epochs[0:idx]
test = temporal_epochs[idx:]
train_labels = np.ones(len(train)).reshape(-1) * 4
test_labels = np.ones(len(test)).reshape(-1) * 4
    

In [19]:
#ige_train, ige_test, ige_train_labels, ige_test_labels = IGE('D:\EEG\IGE', seglength = 9000, train_ratio = 0.75)
#focal_train, focal_test, focal_train_labels, focal_test_labels = FOCAL('D:\EEG\FOCAL', seglength= 10000)
#pnes_train, pnes_test, pnes_train_labels, pnes_test_labels = PNES('D:\EEG\PNES')

temporal_train, temporal_test, temporal_train_labels, temporal_test_labels = TLE('D:\EEG\TLE', train_ratio = 0.8, seglength = 9000)



In [24]:
temporal_train.shape

(2562, 9000, 20)